# Piping Chains and the RunnablePassthrough Class

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [21]:
RunnablePassthrough().invoke([1, 2, 3])

[1, 2, 3]

In [22]:
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} needs?
Answer only by listing the tools.
''')

chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [23]:
chat_template_tools

ChatPromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, template='\nWhat are the five most important tools a {job title} needs?\nAnswer only by listing the tools.\n'), additional_kwargs={})])

In [24]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

In [25]:
string_parser = StrOutputParser()

In [26]:
chain_tools = (chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()})
chain_strategy = chat_template_strategy | chat | string_parser

chain_tools

ChatPromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, template='\nWhat are the five most important tools a {job title} needs?\nAnswer only by listing the tools.\n'), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 8192, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': False, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x000001D18913A190>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D1891391D0>, root_client=<openai.OpenAI o

In [27]:
print(chain_tools.invoke({'job title':'data scientist'}))

{'tools': '1. Python\n2. R Programming\n3. SQL\n4. Tableau\n5. Hadoop'}


In [28]:
print(chain_strategy.invoke({'tools':'''
1. Python
2. R Programming
3. SQL
4. Tableau
5. Hadoop
'''}))

1. Python: Start with the basics of Python, such as variables, data types, loops, and functions. Once you have a good understanding of these, move on to more advanced topics like classes and objects. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding problems on platforms like LeetCode or HackerRank. Work on small projects to apply what you've learned.

2. R Programming: Start with the basics of R, such as data


In [29]:
chain_combined = chain_tools | chain_strategy

In [30]:
print(chain_combined.invoke({'job title':'data scientist'}))

1. Python: Start with the basics of Python, such as variables, data types, loops, and functions. Once you have a good understanding of these, move on to more advanced topics like classes and objects. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding problems on websites like HackerRank or LeetCode. Work on small projects to apply what you've learned.

2. R Programming: Start with the basics of R, such as data


In [31]:
chain_long = (chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()} | 
              chat_template_strategy | chat | string_parser)
print(chain_long.invoke({'job title':'data scientist'}))

1. Python: Start with the basics of Python, such as variables, data types, loops, and functions. Once you have a good understanding of these, move on to more complex topics like classes and objects. Use online resources like Codecademy, Coursera, or Python's official documentation. Practice coding regularly and work on small projects to apply what you've learned.

2. R Programming: Similar to Python, start with the basics of R. Learn about data structures, loops, and


In [33]:
chain_long.get_graph().print_ascii()    

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       